# 뉴스 기사 내용 유사도 확인하기

<div style="text-align: right"> 컴퓨터 공학과 2021105662 홍예림 </div>

### (1) 주제 선정 이유
    
    뉴스는 신속하게 정치, 사회, 경제 등 여러 분야의 새로운 정보를 얻을 수 있어서 오랫동안 많은 사람이 애용해왔다. 사람들이 종이신문보다 언론 사이트를 더 많이 찾아보게 되면서 문제가 생겨났다. 언론 사이트는 방문자 수가 많을수록 광고를 통한 수익도 상승하기 때문에 비슷한 내용의 기사를 여러 개 작성하여 낚시성의 제목으로 포털 사이트에 노출한다. 그래서 정보를 얻기 위해 뉴스 기사를 읽는다면 계속해서 비슷한 내용의 뉴스 기사만 읽게 된다. 지금까지 인터넷 뉴스를 읽은 경험에 따르면 다른 분야보다 특히 더 연예계, 정치, 경제 분야에서 비슷한 내용의 기사들이 매일 쏟아져 나온다. 최근 네이버의 실시간 검색어가 사라지면서 더 자극적이고 낚시성이 높은 제목을 붙인 기사들이 많아졌다. 하지만 기사 내용은 다른 기사들과 다를 바가 없다는 것을 계속해서 느껴왔다. 그래서 이를 해결하고자 텀 프로젝트 주제로 선정하게 되었다.
    

### (2) 주제 정의
    
    Term 프로젝트를 통해 여러 언론 사이트에서 비슷한 내용의 뉴스 기사를 올리는 것이 사실인지 확인하고자 한다. 하나의 키워드를 골라 네이버에서 검색하면 나오는 오늘 발행된 뉴스 기사를 네이버 오픈 API를 이용하여 크롤링한다. 그다음, 뉴스 기사 제목과 url을 json 파일로 저장한 뒤 url을 하나씩 읽어와서 뉴스 기사 사이트에 있는 기사 부분만 beautifulsoup를 이용해 크롤링한다. 이후 konlpy를 이용해 형태소 분석을 해서 뉴스 기사에 있는 명사들을 json 파일로 저장한다. 검색된 뉴스 기사 중 하나를 골라 전체 뉴스 기사와 내용의 유사성을 TF-IDF를 이용해 증명할 예정이다.
    

### (3) 인터넷을 통한 데이터 획득 - 1/2
    
   #### 1번 과정)  네이버 api를 이용하여 키워드를 검색하여 나온 뉴스 기사들의 url과 제목을 수집하여 json 파일로 저장
    네이버 오픈 API를 이용해 네이버 뉴스 검색 결과를 받아온다. 비로그인 오픈 API이므로 GET으로 호출할 때 때 HTTP Header에 애플리케이션 등록 시 발급받은 Client ID와 Client Secret 값을 입력한다. 키워드는 사용자의 입력을 받아 설정한다. 키워드를 검색하여 나온 네이버 뉴스 검색 결과를 받아오면, 제목과, url, original url, 언론 사이트에서 제공하는 기사 요약, 발행된 날짜를 얻을 수 있다.
    link가 네이버 뉴스와 연동되는 링크인 news.naver을 포함하고 있으면 뉴스 기사를 저장한다. 이 데이터 중에서 오늘 발행된 기사만 고르기 위해 datetime 모듈을 호출해서 현재 날짜를 받아온다. 네이버 오픈 API에서 받아온 날짜 데이터 순서는 요일, 일, 월, 년도이고, datetime 모듈을 통해 받아온 날짜 데이터 순서는 년도, 월, 일이다. 네이버 오픈 API로 받아온 날짜 데이터를 수정해서 datetime 모듈의 날짜 데이터 순서에 맞춘다. 네이버 오픈 API에서 얻는 발행된 날짜를 오늘 날짜와 비교하여 오늘 발행된 데이터들만 수집한다. 또한 오늘 안에 발행된 기사 중에서도 검색한 시간과 가장 근접한 시간대에 발행된 기사만 모으기 위해 뉴스 기사를 100개만 저장한다. 키워드로 검색하여 나온 뉴스 기사중에서 오늘 발행된 뉴스 기사가 없으면 에러 코드인 404를 리턴한다.
    json 모듈을 이용해 기사 번호와 제목, url을 json 파일로 저장한다. 네이버 오픈 API로 받아온 데이터를 naver_api_original_data.json로 저장하고, 받아온 데이터 중에서 필요한 데이터만 추출해서 naver_api_result.json로 저장한다.  
    

In [19]:

# 1) 네이버 api를 이용하여 키워드를 검색하여 나온 뉴스 기사들의 url과 제목을 수집하여 json 파일로 저장 #


import urllib.request
import json
from datetime import datetime
import codecs


class LinkCollector:

    # 네이버 뉴스 검색 결과를 출력해주는 네이버 API #

    _CLIENT_ID = "hqKMORcNCRx2jN3kSF6J"     # 애플리케이션 등록 시 발급받은 client id 값 #
    _CLIENT_SECRET = "PbexHJlugP"   # 애플리케이션 등록 시 발급받은 client secret 값 #
    _DISPLAY = 100

    # HTTP Header에 애플리케이션 등록 시 발급받은 Client ID와 Client Secret 값을 같이 전송하면 api 활용 가능 #

    def __init__(self, word, date):
        self.search_word = urllib.parse.quote(word)
        self.date = date
        self.start = 1
        self.temp_url = "https://openapi.naver.com/v1/search/news?query={0}&start={1}&display={2}&sort=date"
        self.total = 0
        self._total()
        self.count = 0
        self.filename = './naver_api_result.json'
        self.testfile = './naver_api_original_data.json'
        self.result = dict()    # 뉴스 기사들의 번호, 제목, url을 저장하는 딕셔너리 #
        self.navertest = []     # 네이버 api를 이용해 받아온 데이터 원본을 저장하는 딕셔너리#
        self.collect_link()
        self.save_to_file()

    def _total(self):
        url = self.temp_url.format(self.search_word, self.start, LinkCollector._DISPLAY)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id", LinkCollector._CLIENT_ID)
        request.add_header("X-Naver-Client-Secret", LinkCollector._CLIENT_SECRET)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        try:
                    response = urllib.request.urlopen(request)
                    rescode = response.getcode()

                    if rescode == 200:
                        response_body = response.read()
                        r = response_body.decode('utf-8')
                        r = json.loads(r)
                        self.total = r.get('total')

                    else:

                        # 에러 코드 확인 #

                        print("Error Code:" + rescode)
                        self.total = 0

        except:
            return

    def collect_link(self):
        is_stop = False

        for i in range(0, int(self.total / 100)):
            self.start = i * LinkCollector._DISPLAY + 1
            url = self.temp_url.format(self.search_word, self.start, LinkCollector._DISPLAY)

            request = urllib.request.Request(url)
            request.add_header("X-Naver-Client-Id", LinkCollector._CLIENT_ID)
            request.add_header("X-Naver-Client-Secret", LinkCollector._CLIENT_SECRET)
            try:
                response = urllib.request.urlopen(request)
                rescode = response.getcode()

                if rescode == 200:
                    response_body = response.read()
                    r = response_body.decode('utf-8')
                    r = json.loads(r)

                    for v in r.get('items'):
                        self.navertest.append(v)
                        title = '{0}'.format(v.get('title'))     # 네이버 api로 얻어온 정보 중에서 제목 부분 추출 #
                        link = v.get('link')        # 네이버 api로 얻어온 정보 중에서 link 부분 추출 #

                        pub_date = v.get('pubDate')     # 네이버 api로 얻어온 정보중에서 날짜 부분 추출 #
                        pub_date = pub_date.split()[:4] # 요일, 일, 월, 년도, 시간 부분만 추출 #
                        pub_date = ' '.join(pub_date)

                        # datetime 모듈로 불러온 date 데이터를 요일, 일, 월, 년도, 시간 순으로 변경하기 #

                        date_obj = datetime.strptime(pub_date, '%a, %d %b %Y')
                        date_obj = date_obj.date()

                        if '//news.naver.com/main/' not in link:    # 네어버 뉴스로 연결되는 뉴스로만 데이터 가져오기 #
                            continue

                        if self.date != date_obj or self.count+1 > 100:   # 네이버 api로 얻어온 정보의 날짜가 오늘 날짜와 다르면 끝내기 #
                            is_stop = True
                            break
                        self.count += 1
                        self.result[self.count] = {'link': link, 'title': title}
                        
                if is_stop:
                    break
            except:
                continue
    
    # 키워드를 입력했을 때 검색된 결과가 없을 경우 404를 리턴함 #
    
    def news_none(self):     
        if len(self.result) == 0:
            return  404


    def save_to_file(self):

        # 데이터 json 파일로 저장하기 #
        
        print("*"*100)
        print("\n뉴스 기사 제목과 링크 크롤링이 완료되었습니다. 잠시만 기다려주세요.")


        json_object1 = json.dumps(self.result, ensure_ascii = False)
        with codecs.open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(json_object1, f, ensure_ascii = False)

        json_object2 = json.dumps(self.navertest, ensure_ascii = False)
        with codecs.open(self.testfile, 'w', encoding='utf-8') as f:
            json.dump(json_object2, f, ensure_ascii = False)

            

### (3) 인터넷을 통한 데이터 획득 - 2/2

#### 2번 과정)   1 에서 저장한 데이터를 불러와 url에서 뉴스 기사 부분만 추출해 json 파일로 저장
    1번 과정을 통해 얻은 기사 번호와 제목, url이 있는 json 파일을 불러와 검색된 기사를 하나씩 읽어드린다. 기사 내용이 있는 부분을 파싱하기 위해 네이버 뉴스의 기사 내용 부분의 공통된 태그를 찾아 파싱한다. 대부분의 뉴스의 기사 내용 부분의 태그는 #articleBodyContents 이고, 사진이 포함된 뉴스의 기사 내용 부분의 태그는 #articeBody 이다. 파싱한 뉴스의 기사 내용 부분을 딕셔너리로 만들어 final_result.json 파일로 저장한다.
    

In [20]:
# 2) 1에서 저장한 데이터를 불러와 url에서 뉴스 기사 부분만 추출해 json 파일로 저장 #

import json
import codecs
import requests
from bs4 import BeautifulSoup


class ContentCollect:
    def __init__(self):
        self.filename = './naver_api_result.json'
        f = open(self.filename, encoding='utf-8')   # 1에서 얻은 뉴스 기사들의 번호, 제목, url가 저장된 json 파일 열기 #
        self.temp = json.loads(json.load(f))
        self.content = dict()
        self.file_name = './final_result.json'
        self.count = 0
        

    def run(self):
        for i, v in self.temp.items():
            link = v.get('link')
            request_headers = { 
            'User-Agent' : ('Mozilla/5.0 (Windows NT 10.0;Win64; x64)\
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98\
            Safari/537.36'), } 

            try:
                response = requests.get(link, headers=request_headers)  # 저장된 link 불러오기 #
            except:
                print('*' * 100, link)
                continue 
            
            # link에 들어가서 뉴스 기사 내용 있는 부분만 파싱 #
            
            soup  = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
            response.encoding = None
            main = soup.select('#articleBodyContents')

            if main == []:
                main = soup.select('#articeBody')
            text = main[0].text
            self.content[i] = text
    
    def save_to_file(self):
        
        # 뉴스 기사 내용 부분만 json 파일로 저장 #
        
        print("*"*100)
        print("\n뉴스 기사 내용 크롤링이 완료되었습니다. 잠시만 기다려주세요.")
        
        json_object = json.dumps(self.content, ensure_ascii = False)
        with codecs.open(self.file_name, 'w', encoding='utf-8') as f:
            json.dump(json_object, f, ensure_ascii = False)
        


### (4) 분석을 위한 데이터의 가공 - 1/2
      
#### 3번 과정)   2에서 추출한 뉴스 기사들을 형태소로 나눈 뒤 명사 따로 json 파일로 저장
    2번 과정을 통해 얻는 뉴스의 기사 내용 부분을 불러와 konlpy를 이용하여 형태소 분석을 한다. konlpy 라이브러리가 설치되어 있지 않으면 설치되어 있지 않다는 에러 메시지를 출력하고 404라는 에러 코드를 리턴한다. konlpy에 있는 kkma 클래스를 이용해 뉴스 기사에 있는 명사만 추출한다. 뉴스 번호와 뉴스에 있는 명사를 딕셔너리로 만들어서 형태소 분리 후 기사에 있는 명사.json으로 저장한다.
    

In [21]:
# 3) 2에서 추출한 뉴스 기사들을 형태소로 나눈 뒤 명사 따로 json 파일로 저장 #

from konlpy.tag import Kkma
import json
import codecs

class MorphemeAnalysis:
    def __init__(self):
        self.openfilename = './final_result.json'
        f = open(self.openfilename, encoding='utf-8')     # 2에서 얻은 뉴스 기사 내용 부분만 저장된 json 파일 열기 #
        self.temp = json.loads(json.load(f))
        self.filename = './형태소 분리 후 기사에 있는 명사.json'
        self.kkma = None
        self.noun_dict = dict()     # 기사에 있는 명사가 모여있는 딕셔너리 #


    def run(self):
        noun_list = []
        try:
            self.kkma = Kkma()
        except:
            print("*"*100)
            print("Konlpy 라이브러리가 설치되어있지 않습니다.")
            print("*"*100)

            return 404       
            
        for i, v in self.temp.items():
            noun_list.clear()
            morph = self.kkma.nouns(v)   # 기사에 있는 명사만 konlpy의 kkma를 이용해 추출 #
            nouns = ' '
      
            for noun in morph:
                nouns += noun + ' '
            
            noun_list.append(nouns)
            self.noun_dict[i] = noun_list.copy()


    def save_to_file(self):
        
        # 뉴스 기사에 있는 명사만 json 파일로 저장 #
        
        print("*"*100)
        print("\n형태소 분석이 완료되었습니다. 잠시만 기다려주세요.")
        
        json_object = json.dumps(self.noun_dict, ensure_ascii = False)
        with codecs.open(self.filename, 'w', encoding='utf-8') as f:
            json.dump(json_object, f, ensure_ascii = False)



### (4) 분석을 위한 데이터의 가공 - 2/2
      
#### 4번 과정)  TF-IDF를 이용해 기사끼리 유사도 검사하기
    3번 과정을 통해 얻은 기사에 들어 있는 명사만 저장한 파일을 불러와 전체 뉴스 기사의 유사도를 확인한다. TfidfVectorizer 라이브러리가 설치죄어 있지 않으면 설지되어 있지 않다는 에러 메시지를 출력하고 404라는 에러 코드를 리턴한다. TF-IDF를 이용해 유사도를 검증한다. TF-IDF는 여러 문서로 이루어진 문서군이 있을 때, 문서의 수와 그 문서에 나타난 단어의 빈도수를 분석하여 유사도를 나타낸다. 유사도는 0에서 1 사이의 값을 가진다. 0이면 아예 다른 기사이고, 1이면 완전히 유사한 기사이다. TF-IDF를 통해 나온 결과를 TF-IDF.xlsx으로 저장한다.
    

In [22]:
# 4) TF-IDF를 이용해 기사끼리 유사도 검사하기 #

from sklearn.feature_extraction.text import TfidfVectorizer
import json
import pandas as pd

class TF_IDF:
    def __init__(self):
        self.openfilename = './형태소 분리 후 기사에 있는 명사.json'
        f = open(self.openfilename, encoding='utf-8')   # 3에서 얻은 뉴스 기사에 있는 명사만 저장한 json 파일 열기 #
        self.temp = json.loads(json.load(f))
        self.TF_noun_list = []
        

    def run(self):
        for i, v in self.temp.items():
            self.TF_noun_list.append(v[0])         
        try:
            tfidf_vectorizer = TfidfVectorizer(min_df=1)    # min-df는 최소 빈도값을 설정해주는 파라미터 #

            # 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 Document Term Matrix(문서 단어 행렬) 을 리턴 #

            tfidf_matrix = tfidf_vectorizer.fit_transform(self.TF_noun_list)

            document_distances = (tfidf_matrix * tfidf_matrix.T)
            
            # 유사도 결과는 0에서 1 사이의 값이 나옴 (아예 다르면 0, 똑같으면 1이 나옴) #

            # 기사끼리의 유사도를 excel 파일로 저장 #
            
            print("*"*100)
            print("\n뉴스 기사 유사도 검사가 완료되었습니다. 잠시만 기다려주세요")
            
            df = pd.DataFrame(document_distances.toarray())
            df.to_excel('TF-IDF.xlsx', sheet_name= 'Sheet1')

        except:
            print("*"*100)
            print("TfidfVectorizer 라이브러리가 설치되어있지 않습니다.")
            print("*"*100)
            return 404



### (5) 분석 결과 도출 - 1/2
    
#### 5번 과정)   선택한 기사와 용인되는 수준의 표절률을 가진 기사 추출하기
    4번 과정을 통해 얻은 유사도가 저장된 xlsx파일을 불러온다. 사용자에게 뉴스 기사의 번호와 뉴스 기사의 제목을 출력해 보여준다. 사용자가 유사도를 검사할 뉴스 기사의 번호를 입력받고 용인되는 수준의 유사도 또한 입력받는다. xlsx에 에서 입력받은 번호의 행을 읽어온다. 유사도 값이 0에서 1 사이의 값이므로 100을 곱해서 유사도 결과를 백분위로 나타낸다. 용인되는 수준의 유사도 이상인 유사도 결과를 가진 뉴스를 출력해준다.
    

In [23]:
#5) 선택한 기사와 용인되는 수준의 표절률을 가진 기사 추출하기 #

import json
from openpyxl import load_workbook

class Similarity:
    def __init__(self):
        self.openfilename1 = './naver_api_result.json'
        f = open(self.openfilename1, encoding='utf-8')      # 1에서 얻은 뉴스 기사들의 번호, 제목, url가 저장된 json 파일 열기 #
        self.temp = json.loads(json.load(f))
        self.wb = load_workbook(filename = 'TF-IDF.xlsx')      # 4에서 얻은 기사끼리의 유사도만 저장한 json 파일 열기 #
        self.ws = self.wb[self.wb.sheetnames[0]]


    def run(self):
        num_link_dict = dict()

        # 뉴스 기사의 번호와 제목을 딕셔너리로 저장 #
        for i, v in self.temp.items(): 
            num_link_dict[i] = v.get('title')
            print(i, v.get('title'))

        object_num = int(input("유사도 검사를 진행할 1에서 {0} 사이의 뉴스 기사의 번호를 입력하세요 >> ".format(len(self.temp))))
        compare_num = int(input("용인되는 수준의 유사도는 몇 퍼센트 인가요? >> "))
        print("*"*100)

        similarity_percentage = []
        compare_object = num_link_dict[str(object_num)]     # 유사도 검사를 진행할 뉴스 기사 제목 #

        #유사도 검사를 진행할 뉴스 기사와 다른 기사들의 유사도 검사 결과가 저장된 행 불러오기#
        
        row = self.ws['{0}'.format(object_num + 1)]
        
        for cell in row:
            percentage = cell.value * 100     # 유사도 결과를 백분위로 나타내기 #
            similarity_percentage.append(percentage)
        clear = similarity_percentage.pop(0)    # 뉴스 기사의 번호가 있던 [0]의 요소 없애기#

        display_list = []
        for i, compare in enumerate(similarity_percentage):
            if compare >= compare_num:      # 용인되는 수준의 유사도 이상인 유사도 결과를 가진 뉴스 기사 추출하기 #
                if i+1 == object_num:       # 같은 기사의 유사도 결과는 지나가기 #
                    pass
                else:
                    display_list.append([compare_object, num_link_dict[str(i+1)] , similarity_percentage[i]])


        if len(display_list) > 0:
            for i in range(len(display_list)):
                print("제목이 {0}인 뉴스 기사와 제목이 {1}인 뉴스 기사의 유사도는 {2}% 입니다.".format(display_list[i][0], display_list[i][1], display_list[i][2]))
            print("용인되는 유사도를 넘어선 기사는 총 {0}개 입니다.".format(len(display_list)))
        else:
            print("제목이 {0}인 뉴스 기사와 유사도가 {1}% 이상인 뉴스 기사는 없습니다.".format(compare_object, compare_num))
            


### (5) 분석 결과 도출 - 2/2
    
    main 함수를 이용해 1번 과정부터 5번 과정이 이루어지는 클래스 객체를 생성한다. 각 과정의 순서에 맞게 호출한다. 1번 과정부터 5번 과정까지 순서대로 실행되고 결과를 확인할 수 있다. 
    
    과정 중간중간 에러 코드인 404가 출력될 경우 에러 메시지를 출력한 뒤 진행을 종료한다.
    진행이 종료되는 경우 1) 입력된 키워드로 검색된 오늘 뉴스 기사가 없는 경우
    진행이 종료되는 경우 2) konlpy 라이브러리가 설치되어 있지 않은 경우
    진행이 종료되는 경우 3) TfidfVectorizer 라이브러리가 설치되어 있지 않은 경우

In [24]:
# main

from datetime import datetime

def main():
    date = datetime.now().date()
    search_word = input("키워드를 입력하세요 ex) 코로나, 백신, 유재석, 주식, 코인 >>  ")

    o = LinkCollector(search_word, date)

    if o.news_none() != 404:
        
        o2 = ContentCollect()
        o2.run()
        o2.save_to_file()
    
        o3 = MorphemeAnalysis()
        if o3.run() != 404:
            o3.save_to_file()
            
            o4 = TF_IDF()
            if o4.run() != 404:
                o5 = Similarity()
                o5.run()
            else:
                pass
        else:
            pass    
    else:
        print("입력된 키워드로 검색된 오늘 뉴스 기사가 없습니다. 다른 검색어를 입력하세요 ")
        
if __name__ == '__main__':
    main()

키워드를 입력하세요 ex) 코로나, 백신, 유재석, 주식, 코인 >>  주식
****************************************************************************************************

뉴스 기사 제목과 링크 크롤링이 완료되었습니다. 잠시만 기다려주세요.
****************************************************************************************************

뉴스 기사 내용 크롤링이 완료되었습니다. 잠시만 기다려주세요.
****************************************************************************************************

형태소 분석이 완료되었습니다. 잠시만 기다려주세요.
****************************************************************************************************

뉴스 기사 유사도 검사가 완료되었습니다. 잠시만 기다려주세요
1 네이버 노조 “숨진 직원, 괴롭힘·과로…회사가 묵인하고 방조”
2 주가에 날개 달린 두산중공업..'신중 투자' 경계 나오는 까닭은?
3 맛과 쉼이 공존하는 공간에서 나눔을 실천하다, 이학순베이커리 이학순 대표
4 리서치알음 &quot;1년간 상장사 <b>주식</b> 투자 수익률 -6.8%&quot;
5 유진투자, 美프리·애프터마켓 거래서비스
6 `카카오손보` 이번주 결정… 빅테크 첫 보험진출 이뤄질까
7 '카카오손보' 예비허가 9일 결정…빅테크 보험업 진출 성사되나
8 日·필리핀 이중국적 사소 US여자오픈 우승에 일본 환호
9 3년전 대법 판결 뒤집혔다…'日징용 소송' 각하
10 최대규모 강제징용 손배소, 1심서 원고패소…&quot;인용하면 국제법 위반&quot;(재종합)
11 ‘비즈니스 리뷰’ 1주년, 월가 역사 돌아본다


### (6) 결론
    
    이 프로그램을 실행하면 사용자가 입력한 키워드로 검색된 뉴스 기사들을 확인할 수 있다. 검색된 뉴스 기사들의 제목을 보고 하나의 뉴스 기사를 선택하고 용인할 수 있는 유사도 값을 입력한다. 그러면 프로그램 실행 결과로 선택한 하나의 뉴스 기사와 용인할 수 있는 유사도 값을 넘어선 뉴스 기사들의 제목과 유사도가 출력된다. 
    프로그램 결과를 통해 기사 내용이 비슷한 기사들을 확인할 수 있고 뉴스를 통해 정보를 얻을 때 비슷한 뉴스 기사는 한 번만 확인하여 더욱 빠르고 정확한 정보를 얻을 수 있게 된다. 어떤 키워드를 입력하느냐에 따라 뉴스 기사들의 유사도 값이 달라지지만 현재 이슈가 되는 코로나, 백신과 같은 키워드 또는 연예계, 정치, 주식과 관련된 키워드를 입력 했을때 기사들의 유사도가 높게 나왔다.
    

### (7) 참고문헌
    
    출처: 네이버 api 호출 예제
          박은정, 조성준, 'KoNLPy: 쉽고 간결한 한국어 정보처리 파이썬 패키지', 제 26회 한글 및 한국어 정보처리 학술대회 논문집, 2014.
          유원준, 딥러닝을 이용한 자연어 처리 입문, 2021

### (8) 별첨 (생성된 파일)
    naver_api_original_data.json : 네이버 api를 이용해 획득한 데이터 원본
    naver_api_result.json : 가공된 네이버 api 데이터
    final_result.json : 크롤링을 이용해 획득한 기사 본문 원본
    형태소 분리 후 기사에 있는 명사.json : 가공된 기사 본문 데이터 
    TF-IDF.xlsx : 가공된 기사 본문 데이터를 가지고 검사한 유사도 결과 데이터